In [ ]:
#!pip install -q transformers

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import zipfile

In [6]:
# Sample 5000 rows from all training sets for computation purposes
train_data_w = dataframes['train_data_w']
train_data_m = dataframes['train_data_m']
train_data_q = dataframes['train_data_q']

train_w_samp = train_data_w.sample(n=5000, random_state=0)
train_m_samp = train_data_m.sample(n=5000, random_state=1)
train_q_samp = train_data_q.sample(n=5000, random_state=2)

In [7]:
# Sample 1000 rows from all testing sets for computation purposes
test_data_w = dataframes['test_data_w']
test_data_m = dataframes['test_data_m']
test_data_q = dataframes['test_data_q']

test_w_samp = test_data_w.sample(n=1000, random_state=3)
test_m_samp = test_data_m.sample(n=1000, random_state=4)
test_q_samp = test_data_q.sample(n=1000, random_state=5)

In [8]:
# Extract text
train_w_text = list(train_w_samp['body'].to_numpy(dtype=str))
train_m_text = list(train_m_samp['body'].to_numpy(dtype=str))
train_q_text = list(train_q_samp['body'].to_numpy(dtype=str))

test_w_text = list(test_w_samp['body'].to_numpy(dtype=str))
test_m_text = list(test_m_samp['body'].to_numpy(dtype=str))
test_q_text = list(test_q_samp['body'].to_numpy(dtype=str))

In [ ]:
# Get sentiment labels
# from transformers import pipeline
# sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

# train_w_labels = sentiment_pipeline(train_w_text)
# train_m_labels = sentiment_pipeline(train_m_text)
# train_q_labels = sentiment_pipeline(train_q_text)
# print("Training done")

# test_w_labels = sentiment_pipeline(test_w_text)
# test_m_labels = sentiment_pipeline(test_m_text)
# test_q_labels = sentiment_pipeline(test_q_text)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Training done


In [40]:
train_w_labels = np.load("train_w_labels.npy", allow_pickle=True)
train_m_labels = np.load("train_m_labels.npy", allow_pickle=True)
train_q_labels = np.load("train_q_labels.npy", allow_pickle=True)

test_w_labels = np.load("test_w_labels.npy", allow_pickle=True)
test_m_labels = np.load("test_m_labels.npy", allow_pickle=True)
test_q_labels = np.load("test_q_labels.npy", allow_pickle=True)

In [43]:
test_m_labels

array([{'label': 'neutral', 'score': 0.8489762544631958},
       {'label': 'neutral', 'score': 0.8424960970878601},
       {'label': 'positive', 'score': 0.8533715009689331},
       {'label': 'neutral', 'score': 0.9255430698394775},
       {'label': 'negative', 'score': 0.5636560916900635},
       {'label': 'neutral', 'score': 0.5548813939094543},
       {'label': 'neutral', 'score': 0.8414508104324341},
       {'label': 'neutral', 'score': 0.5910891890525818},
       {'label': 'negative', 'score': 0.8839585781097412},
       {'label': 'negative', 'score': 0.6898341774940491},
       {'label': 'neutral', 'score': 0.8501808047294617},
       {'label': 'neutral', 'score': 0.6298134326934814},
       {'label': 'neutral', 'score': 0.652154266834259},
       {'label': 'neutral', 'score': 0.8017043471336365},
       {'label': 'neutral', 'score': 0.9031584858894348},
       {'label': 'negative', 'score': 0.5752916932106018},
       {'label': 'neutral', 'score': 0.792360246181488},
       {'la

In [44]:
def add_sentiments_df(df, sentiments):
    pos_indicator = []
    neut_indicator = []
    neg_indicator = []
    scores = []

    for tweet in sentiments:
        if tweet['label'] == 'positive':
            pos_indicator.append(1)
            neut_indicator.append(0)
            neg_indicator.append(0)
        elif tweet['label'] == 'neutral':
            pos_indicator.append(0)
            neut_indicator.append(1)
            neg_indicator.append(0)
        else:
            pos_indicator.append(0)
            neut_indicator.append(0)
            neg_indicator.append(1)
        scores.append(tweet['score'])

    scores_pos_prod = np.multiply(pos_indicator, scores)
    scores_neut_prod = np.multiply(neut_indicator, scores)
    scores_neg_prod = np.multiply(neg_indicator, scores)

    new_columns_data = {'I(positive)': pos_indicator,
                    'I(neutral)': neut_indicator,
                    'I(negative)': neg_indicator,
                    'I(positive)*score': scores_pos_prod,
                    'I(neutral)*score': scores_neut_prod,
                    'I(negative)*score': scores_neg_prod}

    new_columns_df = pd.DataFrame(new_columns_data)
    df_updated = pd.concat([df.reset_index(), new_columns_df], axis=1)

    return df_updated

In [45]:
train_w_samp_updated = add_sentiments_df(train_w_samp, train_w_labels)
train_m_samp_updated = add_sentiments_df(train_m_samp, train_m_labels)
train_q_samp_updated = add_sentiments_df(train_q_samp, train_q_labels)

test_w_samp_updated = add_sentiments_df(test_w_samp, test_w_labels)
test_m_samp_updated = add_sentiments_df(test_m_samp, test_m_labels)
test_q_samp_updated = add_sentiments_df(test_q_samp, test_q_labels)

In [46]:
train_w_samp_updated.to_csv("train_w_samp_sentiments.csv")
train_m_samp_updated.to_csv("train_m_samp_sentiments.csv")
train_q_samp_updated.to_csv("train_q_samp_sentiments.csv")

test_w_samp_updated.to_csv("test_w_samp_sentiments.csv")
test_m_samp_updated.to_csv("test_m_samp_sentiments.csv")
test_q_samp_updated.to_csv("test_q_samp_sentiments.csv")

In [30]:
test_w_samp_updated

,index,body,Target,Date,I(positive),I(neutral),I(negative),I(positive)*score,I(neutral)*score,I(negative)*score
0,6252,light one up elon tesla,1,2019-12-23 16:05:42,0,1,0,0.000000,0.819609,0.000000
1,4684,great call i close my last tesla call at 339 t...,1,2019-11-07 16:08:36,1,0,0,0.973899,0.000000,0.000000
2,1731,those are completely different beastswith its ...,0,2019-04-29 01:24:45,0,1,0,0.000000,0.739621,0.000000
3,4742,every little pumpleads to a little dumptsla,1,2019-05-29 18:22:09,0,1,0,0.000000,0.483492,0.000000
4,4521,dont be surprised even those who placed their ...,0,2019-11-23 16:13:14,0,0,1,0.000000,0.000000,0.914133
...,...,...,...,...,...,...,...,...,...,...
995,3921,dont worry truth well fight shawty until you a...,0,2019-10-23 03:53:55,0,1,0,0.000000,0.650419,0.000000
996,6685,12 i thought the definition of ponzi was a sch...,1,2019-09-05 02:18:41,0,1,0,0.000000,0.720200,0.000000
997,3194,the us18 billion electriccar bubble at risk of...,0,2019-04-16 00:25:55,0,1,0,0.000000,0.491852,0.000000
998,1941,all these tesla cultists are about to get test...,1,2019-04-25 14:10:40,0,0,1,0.000000,0.000000,0.875483


In [47]:
def run_log_reg(df_train, df_test):
    X_train = df_train[['I(positive)', 'I(neutral)', 'I(negative)', 'I(positive)*score', 'I(neutral)*score', 'I(negative)*score']]
    y_train = df_train['Target']

    X_test = df_test[['I(positive)', 'I(neutral)', 'I(negative)', 'I(positive)*score', 'I(neutral)*score', 'I(negative)*score']]
    y_test = df_test['Target']

    clf = LogisticRegression(random_state=0).fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    classification_rep = classification_report(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    print("Classification Report:")
    print(classification_rep)
    
    print("Confusion Matrix:")
    print(conf_matrix)

In [48]:
run_log_reg(train_w_samp_updated, test_w_samp_updated)

Classification Report:
              precision    recall  f1-score   support

           0       0.52      0.77      0.62       495
           1       0.57      0.29      0.38       505

    accuracy                           0.53      1000
   macro avg       0.54      0.53      0.50      1000
weighted avg       0.54      0.53      0.50      1000

Confusion Matrix:
[[383 112]
 [358 147]]


In [49]:
run_log_reg(train_m_samp_updated, test_m_samp_updated)

Classification Report:
              precision    recall  f1-score   support

           0       0.49      0.68      0.57       495
           1       0.50      0.32      0.39       505

    accuracy                           0.49      1000
   macro avg       0.50      0.50      0.48      1000
weighted avg       0.50      0.49      0.48      1000

Confusion Matrix:
[[335 160]
 [345 160]]


In [50]:
run_log_reg(train_q_samp_updated, test_q_samp_updated)

Classification Report:
              precision    recall  f1-score   support

           0       0.49      0.63      0.55       495
           1       0.50      0.36      0.41       505

    accuracy                           0.49      1000
   macro avg       0.49      0.49      0.48      1000
weighted avg       0.49      0.49      0.48      1000

Confusion Matrix:
[[312 183]
 [325 180]]
